<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>


# Google Sheets - Add new GitHub member to team from spreadsheet

**Tags:** #github #teams #automation #googlesheets


**Author:** [Sanjeet Attili](https://linkedin.com/in/sanjeet-attili-760bab190/)


## Input

Pre-requisite:
* Please share your Google Sheet with our service account
For the driver to fetch the contents of your google sheet, you need to share it with the service account linked with Naas. 🔗 naas-share@naas-gsheets.iam.gserviceaccount.com 
* Sample format of the google sheet [looks in this fashion](https://docs.google.com/spreadsheets/d/10VBjpuFy0yAYzWHkK1QetEeqNjkURYKanR865m1FE-I/edit?usp=sharing)
With default column names as 'User_profiles' (user github links), and 'Status_added' (either yes or no)
* If the columns names have to be changed, then please have the first column reserved for github profile links, 
and second column is regarding status of profile

### Import libraries

In [21]:
import requests
from naas_drivers import github, gsheet
import naas

### Setup Email params

In [22]:
email_to = "your@email.com"
subject = "Message from naas - Github Teams profile does not exist!"
content = 'This is a mail for adding new profile to the organization, so that the member can be added to github teams in future automatically!'

### Setup Google Sheet

In [23]:
# Enter spreadsheet_id and sheet name containing members
spreadsheet_id = "****"
sheet_name = "YOUR_SHEET"

### Setup Github
**How to find your personal access token on Github?**

- First we need to create a personal access token to get the details of our organization from [here](https://github.com/settings/tokens)
- You will be asked to select scopes for the token. Which scopes you choose will determine what information and actions you will be able to perform against the API.
- You should be careful with the ones prefixed with write:, delete: and admin: as these might be quite destructive.
- You can find description of each scope in docs [here](https://docs.github.com/en/developers/apps/building-oauth-apps/scopes-for-oauth-apps).

In [24]:
# GitHub token
GITHUB_TOKEN = "ENTER_YOUR_GITHUB_TOKEN_HERE" # EXAMPLE : "ghp_fUYP0Z5i29AG4ggX8owctGnHU**********" 

# GitHub teams url
GITHUB_TEAMS_URL = "ENTER_YOUR_GITHUB_TEAMS_URL_HERE" # EXAMPLE : "https://github.com/orgs/jupyter-naas/teams/opensource-contributors"

### Setup Naas

In [ ]:
#Schedule the notebook to run every day at 09:00 hrs
naas.scheduler.add(cron="0 9 * * *")

# To delete your scheduler, uncomment the line below and execute the cell 
# naas.scheduler.delete()

## Model

In [25]:
#Connect with Gsheet and get all data in a dataframe
gsheet.connect(spreadsheet_id)
df_gsheet = gsheet.get(sheet_name=sheet_name)
df_gsheet.head()

,User_profiles,Status_added
0,https://github.com/Dr0p42,no
1,https://github.com/FlorentLvr,no
2,https://github.com/jravenel,no


**Team profiles available currently**

In [26]:
df_teams = github.connect(GITHUB_TOKEN).teams.get_profiles(GITHUB_TEAMS_URL)
df_teams.head()

,TEAM,SLUG,TEAM_DESCRIPTION,member_profile,GITHUB,NAME,EMAIL,LOCATION,ORGANIZATION,BIO,LOGIN_NAME,TWITTER,CREATED_AT,UPDATED_AT
0,next_gen_coders,next_gen_coders,All cool stuff over here,https://api.github.com/users/SanjuEpic,sanjeet-experiment,Sanjeet Attili,NaN,"Raipur, India",IIIT Naya Raipur,"NLP Enthusiast, OSC@naas.ai",SanjuEpic,NaN,2019-11-10T09:47:56Z,2022-05-12T02:54:28Z
1,random1,random1,a random expt channel,https://api.github.com/users/SanjuEpic,sanjeet-experiment,Sanjeet Attili,NaN,"Raipur, India",IIIT Naya Raipur,"NLP Enthusiast, OSC@naas.ai",SanjuEpic,NaN,2019-11-10T09:47:56Z,2022-05-12T02:54:28Z


In [27]:
def add_members_from_sheet(TEAMS_URL, df_gsheet, df_teams):
    
    # Format of url = "https://api.github.com/orgs/jupyter-naas/members"
    ORG_MEMBERS_URL = "api.github.com".join(TEAMS_URL.split('/teams/')[0].split('github.com'))+ '/members'

    team_name = TEAMS_URL.split('/teams/')[-1]

    headers = github.connect(GITHUB_TOKEN).headers
    
    # extracting the slug name of the team
    team_slug = df_teams[df_teams['TEAM'] == team_name]['SLUG'][0]
    profiles_col, status_col = df_gsheet.columns.to_list()[0], df_gsheet.columns.to_list()[1]
    
    # Getting the members github profile link to add them to a team
    member_details = requests.get(ORG_MEMBERS_URL, headers = headers).json()

    org_members = [member['login'] for member in member_details]
    
    # Format of url for adding member to team -> https://api.github.com/orgs/{org}/teams/{team_slug}/memberships/{username}
    URL = "api.github.com".join(TEAMS_URL.split('/teams/')[0].split('github.com')) +\
    '/teams/'+f'{team_slug}'+'/memberships/'
    
    profiles_in_sheet = [profile.split('github.com/')[-1] for profile in df_gsheet[profiles_col].to_list()]
    
    for idx, handle in enumerate(profiles_in_sheet):
        if handle in org_members:

            if df_gsheet.loc[idx, status_col] == 'yes':
                continue

            r = requests.put(url = URL+handle,
                             headers = headers)
            if r.status_code == 200:
                print(f'Successfully added the member {handle} to team {team_slug}')
                df_gsheet.loc[idx, status_col] = 'yes'

            else:
                print(r)
        else:
            print(f'Member not found with the given profile, a mail has been sent to add {handle} to the organization')
            naas.notification.send(email_to = email_to, subject = subject, html = content + f"\n<a href = https://github.com/{handle}> Click here to view profile </a>")
            df_gsheet.loc[idx,status_col] = 'Profile does not exist, mail sent to the team!'
        
    #Update the spreadsheet
    gsheet.connect(spreadsheet_id).send(
    sheet_name=sheet_name,
    data=df_gsheet,
    append=False)
    
    return df_gsheet

df_gsheet = add_members_from_sheet(GITHUB_TEAMS_URL, df_gsheet, df_teams)
df_gsheet.head()

Successfully added the member Dr0p42 to team next_gen_coders
Successfully added the member FlorentLvr to team next_gen_coders
Member not found with the given profile, a mail has been sent to add jravenel to the organization
👌 💌 Email has been sent successfully !


,User_profiles,Status_added
0,https://github.com/Dr0p42,yes
1,https://github.com/FlorentLvr,yes
2,https://github.com/jravenel,"Profile does not exist, mail sent to the team!"


## Output

**Now let us see the updated list of profiles corresponding to the teams**

In [28]:
df_teams = github.connect(GITHUB_TOKEN).teams.get_profiles(GITHUB_TEAMS_URL)
df_teams.head()

,TEAM,SLUG,TEAM_DESCRIPTION,member_profile,GITHUB,NAME,EMAIL,LOCATION,ORGANIZATION,BIO,LOGIN_NAME,TWITTER,CREATED_AT,UPDATED_AT
0,next_gen_coders,next_gen_coders,All cool stuff over here,https://api.github.com/users/Dr0p42,sanjeet-experiment,Maxime Jublou,maxime@jublou.fr,Pyrénées,naas.ai,PGP Key: https://keybase.io/jubloum,Dr0p42,NaN,2016-09-22T08:07:56Z,2022-04-14T04:59:18Z
1,next_gen_coders,next_gen_coders,All cool stuff over here,https://api.github.com/users/FlorentLvr,sanjeet-experiment,None,None,None,None,None,FlorentLvr,NaN,2019-02-26T20:34:19Z,2022-04-20T06:35:32Z
2,next_gen_coders,next_gen_coders,All cool stuff over here,https://api.github.com/users/SanjuEpic,sanjeet-experiment,Sanjeet Attili,None,"Raipur, India",IIIT Naya Raipur,"NLP Enthusiast, OSC@naas.ai",SanjuEpic,NaN,2019-11-10T09:47:56Z,2022-05-12T02:54:28Z
3,random1,random1,a random expt channel,https://api.github.com/users/SanjuEpic,sanjeet-experiment,Sanjeet Attili,None,"Raipur, India",IIIT Naya Raipur,"NLP Enthusiast, OSC@naas.ai",SanjuEpic,NaN,2019-11-10T09:47:56Z,2022-05-12T02:54:28Z


**Updated google sheet below**

In [29]:
gsheet.connect(spreadsheet_id)
df_gsheet = gsheet.get(sheet_name=sheet_name)
df_gsheet

,User_profiles,Status_added
0,https://github.com/Dr0p42,yes
1,https://github.com/FlorentLvr,yes
2,https://github.com/jravenel,"Profile does not exist, mail sent to the team!"
